In [1]:
import pyarrow.parquet as pq
from fuzzywuzzy import fuzz

In [2]:
def get_cols_only(f):
    table = pq.read_table(f)
    return table.schema.names


def get_all_cols(data_ref):
    assert pd.notnull(data_ref)
    out = {}
    for k,v in data_ref.items():
        out[k] = get_cols_only(v)
    return out


def fuzzy_match(l, r, thresh):
    return fuzz.ratio(l, r) >= thresh


def comp_col_list(col_list, col, thresh):
    for v in col_list:
        if fuzzy_match(col, v, thresh):
            return v
    return None


def make_fuzzy_cols(data_ref, thresh=70):
    groups = get_all_cols(data_ref)
    core_cols = groups['orig']
    groups.pop('orig')
    out = {col:[] for col in core_cols}
    for label,other_col_list in groups.items():
        for col in other_col_list:
            # if no exact match for col name, compare col name to known existing
            if col not in out:
                similar = comp_col_list(core_cols, col, thresh)
                if similar:
                    if col not in out[similar]:
                        out[similar].append(col)
                else:
                    out[col] = []
    return out